# Image Classification using PyTorch and 🔭 Galileo

In this tutorial, we'll train a model with PyTorch and explore the results in Galileo.

This notebook pulls data from S3 and is the suggested way for working with images in Galileo.

**Make sure to select GPU in your Runtime! (Runtime -> Change Runtime type)**

# 1. Install Prerequisites and Login Galileo

In [ ]:
#@markdown Install `dataquality`
# Upgrade pip
!pip install -U pip &> /dev/null

# Install all dependecies
!pip install -U dataquality &> /dev/null

print('👋 Installed necessary libraries!')

In [ ]:
#@markdown Check that a GPU is available

import torch
# Check Cuda.
if torch.cuda.is_available():
  print("⚡ You are connected to a GPU!")
else:
  print("❗You are NOT connected to a GPU ❗It is recommended to connect to a GPU before training")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
import dataquality as dq

dq.login()

# 2. Download the data in the notebook

Pull the data from GCS or S3. 

For enterprise customers, your cluster will have to have permissions to request data from S3 and GCS for AWS and GCP clusters. Cross account transfer is not currently supported.

For free users, the data has to be publicly available.

In [ ]:
CLOUD_ZIP_PATH = f"https://storage.googleapis.com/galileo-public-data/CV_datasets/ImageNet10_animals_train_val.zip"

In [ ]:
#@markdown Download the images

LOCAL_DATA_DIR = "dataset"
dataset_dir_name = CLOUD_ZIP_PATH.split('/')[-1].split('.zip')[0]

cmd = f"""
mkdir -p {LOCAL_DATA_DIR}
if [ ! -d {LOCAL_DATA_DIR}/{dataset_dir_name} ]
then
  echo "Downloading data"
  curl {CLOUD_ZIP_PATH} -o {LOCAL_DATA_DIR}/{dataset_dir_name}.zip
  unzip {LOCAL_DATA_DIR}/{dataset_dir_name}.zip -d {LOCAL_DATA_DIR}
else
  echo "Data already exists locally. Moving on."
fi
"""
with open('download_images.sh', 'w') as file:
  file.write(cmd)

!bash download_images.sh

# 3. Initialize Galileo


In [ ]:
# 🔭🌕 DATASET_NAME is only used for creating a run name in Galileo
DATASET_NAME = "ImageNet10_animals_train_val" 

# 🔭🌕 Initializing a new run in Galileo. Each run is part of a project.
dq.init(task_type="image_classification", 
        project_name="image_classification_pytorch", 
        run_name=f"example_run_{DATASET_NAME.replace('/', '-')}")

# 4. Create Dataset and Log Input Data with Galileo

Input data is logged via `log_image_dataset`. This step will log the images, gold labels, data split, and list of all labels. You can achieve by simply calling the method in 1 line of code, see below.

To skip uploading the images, provide their location in the variable `imgs_remote_location`. Note that the images have to be in an (unzipped) folder in the cloud, and the folder structure has to be the same as the local folder structure (in this notebook).

In [ ]:
#@markdown Fix a random Seed and load helper methods.
from typing import Optional, List
from io import BytesIO
from PIL import Image
import numpy as np
import random

# Fix a random seed.
def seed_all(seed: int) -> None:
    """Set all relevant seed for training a Pytorch Model.

    Based on the following post:
    https://discuss.pytorch.org/t/reproducibility-with-all-the-bells-and-whistles/81097
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def seed_worker(worker_id: int) -> None:
    """Set seed for dataloader worker.

    Based on the following post:
    https://discuss.pytorch.org/t/reproducibility-with-all-the-bells-and-whistles/81097
    """
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [ ]:
# Create the Dataset and Dataloader + Log input to 🔭🌕 Galileo

# Create Dataset
import os
from torchvision.datasets import ImageFolder
from torchvision import transforms

image_crop_size = [224, 224]

transform = {
        'train': transforms.Compose([
            transforms.Resize(image_crop_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ]),
        'val': transforms.Compose([
            transforms.Resize(image_crop_size),
            transforms.ToTensor(),
        ])
    }

root_data_dir = os.path.abspath(LOCAL_DATA_DIR)
TRAIN_SPLIT_NAME = "train"
train_dataset = ImageFolder(root=f"{root_data_dir}/ImageNet10_animals_train_val/train", transform=transform["train"])
VAL_SPLIT_NAME = "validation" # this var is needed in dq.set_split down below
val_dataset = ImageFolder(root=f"{root_data_dir}/ImageNet10_animals_train_val/val", transform=transform["val"])

print(f"Loaded {TRAIN_SPLIT_NAME} dataset with {len(train_dataset)} samples and {len(train_dataset.classes)} labels")
print(f"Loaded {VAL_SPLIT_NAME} dataset with {len(val_dataset)} samples and  {len(val_dataset.classes)} labels")

# Fix Labels: The labels of ImageNet are hashes. Convert to human readable labels
CLASSES_DICT = {'n02124075' : 'Egyptian_cat', 'n02107574': 'Greater_Swiss_Mountain_dog', 'n02114367': 'Timber_wolf', 'n02085620': 'Chihuahua', 'n02114548': 'White_wolf', 'n02117135': 'Hyena', 'n02108915': 'French_bulldog', 'n02123159': 'Tiger_cat', 'n02114855': 'Coyote', 'n02106550': 'Rottweiler'}
list_of_labels = [CLASSES_DICT[cls] for cls in train_dataset.classes]
train_dataset.class_to_idx = {lab:i for i, lab in enumerate(list_of_labels)}
val_dataset.class_to_idx = {lab:i for i, lab in enumerate(list_of_labels)}

# 🔭🌕 Galileo log: Set labels
dq.set_labels_for_run(list_of_labels)

# 🔭🌕 Galileo log: Log dataset
dq.log_image_dataset(train_dataset, split=TRAIN_SPLIT_NAME, imgs_remote="gs://galileo-public-data/CV_datasets/ImageNet10_animals_train_val/train")
dq.log_image_dataset(val_dataset, split=VAL_SPLIT_NAME, imgs_remote="gs://galileo-public-data/CV_datasets/ImageNet10_animals_train_val/val")

# Create Dataloader
from torch.utils.data import DataLoader
import torch

BATCH_SIZE = 16
NUM_WORKERS = 0 
SEED_WORKER = 42

seed_all(SEED_WORKER)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, worker_init_fn=seed_worker, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, worker_init_fn=seed_worker, pin_memory=True)

In [ ]:
#@markdown Visualize the Data.
# Visualizing a few images of the dataset (post-processing/augmentation)
import random
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
idxs = [random.randint(0, len(train_dataset) -1) for _ in range(20)]
grid_img = make_grid([train_dataset[idx][0] for idx in idxs], nrow=5)
plt.figure(figsize = (20,10))
plt.imshow(grid_img.permute(1, 2, 0))
plt.show()

# 5. Log model data with Galileo

Model data is logged by wrapping the model with `watch` function. This step will log the model logits and embeddings. You can achieve this by adding 1 line of code to the standard pytorch model. 

In [ ]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(list_of_labels),
    id2label={str(i): c for i, c in enumerate(list_of_labels)},
    label2id={c: str(i) for i, c in enumerate(list_of_labels)},
    ignore_mismatched_sizes = True,
).to(device)

# 🔭🌕 Galileo logging -- Watch model
from dataquality.integrations.torch import watch
watch(
    model=model,
    classifier_layer=model.classifier,
    dataloaders=[train_dataloader, val_dataloader],
)

# 6. Putting into Action: Training a Model

We complete the training pipeline by using a standard PyTorch training setup. To complete logging, we call `dq.finish()` after training.

In [ ]:
from tqdm import tqdm
from time import sleep, time

EPOCHS = 3
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=5e-4)

# Train !
start = time()
print(f"Training for {EPOCHS} epochs on {device}")

for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}/{EPOCHS}")
    dq.set_epoch(epoch)  # 🔭🌕 Galileo -- Set split

    model.train()
    train_loss = torch.tensor(0.0, device=device)
    train_correct = torch.tensor(0, device=device)
    
    dq.set_split(TRAIN_SPLIT_NAME)
    with tqdm(train_dataloader, unit="batch") as train_minibatchs:
        for train_minibatch in train_minibatchs:
            train_minibatchs.set_description(f"Epoch {epoch}")

            images = train_minibatch[0].to(device)
            labels = train_minibatch[1].to(device)

            preds = model(pixel_values=images, labels=labels)
            logits, loss = preds.logits, preds.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                train_loss += loss
                train_batch_correct = (torch.argmax(logits, dim=1) == labels).sum()
                train_correct += train_batch_correct

            train_minibatchs.set_postfix(batch_loss=loss.item(), batch_accuracy=float(train_batch_correct) / BATCH_SIZE)
            sleep(0.001)

    print(f"Training loss: {train_loss:.2f}")
    print(f"Training accuracy: {100 * float(train_correct) / len(train_dataloader.dataset):.2f}")
    
    dq.set_split(VAL_SPLIT_NAME)  # 🔭🌕 Galileo -- Set split
    if val_dataloader is not None:
        model.eval()
        val_loss = torch.tensor(0.0, device=device)
        val_correct = torch.tensor(0, device=device)

        with torch.no_grad():
            for val_minibatch in tqdm(val_dataloader):
                images = val_minibatch[0].to(device)
                labels = val_minibatch[1].to(device)
                
                preds = model(pixel_values=images, labels=labels)
                logits, loss = preds.logits, preds.loss

                val_loss += loss
                val_correct += (torch.argmax(logits, dim=1) == labels).sum()

        print(f"{VAL_SPLIT_NAME} loss: {val_loss:.2f}")
        print(f"{VAL_SPLIT_NAME} accuracy: {100*val_correct/len(val_dataloader.dataset):.2f}")

end = time()
print(f"Total training time: {end-start:.1f} seconds")
dq.finish()

# General Help and Docs
- To get help with your task's requirements, call `dq.get_data_logger().doc()`
- To see more general data and model logging docs, run `dq.docs()`

In [ ]:
dq.get_data_logger().doc()
help(dq.log_dataset)